In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from base_layer_utils import BaseLayerDataRepo, BaseLayerResultsRepo, ModelName
from base_layer_utils import compute_layer1_oof
from base_layer_utils import SklearnBLE

#from fast_text_data import FastTextDataGenerator# fasttext_data_process 
#from tfidf_data import tfidf_data_process

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
import time
import gc
import xgboost as xgb
from sklearn.metrics import roc_auc_score

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
train = pd.read_pickle('/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project1/data/train.pkl').head(10000)
test = pd.read_pickle('/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project1/data/test.pkl')

In [4]:
train_rare = train.copy()
train_rare.loc[train_rare['Month'].value_counts()[train_rare['Month']].values < 800, 'Month'] = 'RARE_VALUE'
train_rare.head(3)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,label
0,RARE_VALUE,19,2,18,21,157,133,6,0
1,4,3,1,12,18,79,172,4,0
2,2,1,6,9,15,129,72,2,0


In [5]:
train.head(2)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,label
0,7,19,2,18,21,157,133,6,0
1,4,3,1,12,18,79,172,4,0


In [6]:
categorical_cols = ["Month", "DayofMonth", "DayOfWeek", "DepTime", "UniqueCarrier", "Origin", "Dest", "Distance"]

for col in categorical_cols:
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

In [7]:
train.shape, test.shape

((10000, 9), (100000, 8))

In [6]:
train.head(2)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,label
0,7,19,2,18,21,157,133,6,0
1,4,3,1,12,18,79,172,4,0


In [7]:
train.dtypes

Month            category
DayofMonth       category
DayOfWeek        category
DepTime          category
UniqueCarrier    category
Origin           category
Dest             category
Distance         category
label               int64
dtype: object

In [8]:
label_cols = ['label']

# tfidf data

In [9]:
bldr = BaseLayerDataRepo()

In [11]:
compatible_models = [ModelName.LGB]#, ModelName.LGB_PERLABEL, ModelName.NBLGB, ModelName.NBLGB_PERLABEL,
#                               ModelName.XGB, ModelName.XGB_PERLABEL, ModelName.NBXGB, ModelName.NBXGB_PERLABEL,
#                               ModelName.LOGREG, ModelName.LOGREG_PERLABEL, ModelName.NBLOGREG, 
#                                 ModelName.NBLOGREG_PERLABEL,
#                               ModelName.LSVC, ModelName.LSVC_PERLABEL, ModelName.NBLSVC, ModelName.NBLSVC_PERLABEL]

bldr.add_tfidf_data(train_sentence=train['comment_text'], test_sentence=test['comment_text'], 
                    y_train=train[label_cols], label_cols=label_cols, 
                    compatible_models=tfidfdata_compatible_models, word_ngram=(1,1), word_max=30000)

In [12]:
feature_cols = list(set(train.columns) - set(label_cols))

In [13]:
X_train = train[feature_cols]
y_train = train[label_cols]#.to_dict('list')
X_test = test[feature_cols]
type(X_train), type(y_train)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [14]:
X_train.shape, y_train.shape

((10000, 8), (10000, 1))

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
rf = RandomForestClassifier(random_state=2018)

In [17]:
rfmodel = rf.fit(X_train, y_train.values.reshape(-1,))

In [18]:
rfmodel.feature_importances_

array([0.15541042, 0.15738642, 0.08627667, 0.1673062 , 0.10196124,
       0.09241364, 0.14263732, 0.09660809])

In [62]:
rfmodel.feature_importances_

array([0.08348223, 0.16801926, 0.10396937, 0.15340707, 0.09553946,
       0.09907577, 0.15402371, 0.14248312])

In [66]:
X_temp = X_train.head(10)

In [68]:
y_temp = y_train.head(10)

In [69]:
from sklearn.model_selection import KFold, StratifiedKFold

In [71]:
kf = KFold(n_splits=5, random_state=2018)

In [75]:
X_train.dtypes

DayOfWeek        int64
DepTime          int64
UniqueCarrier    int64
Origin           int64
Distance         uint8
Month            int64
Dest             int64
DayofMonth       int64
dtype: object

In [77]:
for i, (tr_index, te_index) in enumerate(kf.split(X_temp, y_temp)):
    print(tr_index, te_index)
    print(X_temp.iloc[tr_index])
    break

[2 3 4 5 6 7 8 9] [0 1]
   DayOfWeek  DepTime  UniqueCarrier  Origin  Distance  Month  Dest  \
2          6        9             15     129         2      2    72   
3          6        9             15     147         7      3   220   
4          2       13             13      80         2      4   155   
5          5       15             10      19         3      3   273   
6          6       12             20      48         3      3    86   
7          5        0              4     204         9      1   147   
8          7       19             16     205         4      6   290   
9          1        9              1     214         7      7    80   

   DayofMonth  
2           1  
3          19  
4          28  
5          17  
6          10  
7          14  
8          13  
9          18  


In [56]:
rfmodel.n_estimators

10

In [21]:
bldr.add_data('flight_data_1', X_train, X_test, y_train, label_cols, compatible_models)

In [23]:
print(bldr)

data_id: flight_data_1        
	x_train: (10000, 8)	x_test: (100000, 8)
	y_train type: <class 'dict'>
	compatible_models: {<ModelName.LGB: 5>}
 


In [32]:
temp = bldr.get_data('flight_data_1')

In [33]:
temp

{'compatible_models': {<ModelName.LGB: 5>},
 'data_id': 'flight_data_1',
 'labes_cols': ['label'],
 'x_test':        DayOfWeek  DepTime  UniqueCarrier  Origin  Distance  Month  Dest  \
 0              3        6             22     191         5     10   215   
 1              2        7             20     152         8      7   126   
 2              7        6             13     117         5      4   206   
 3              7       16             20      48         3      6   182   
 4              3       15             18     205         1      9   264   
 5              4        8             14     206         4      5    86   
 6              1        7              4     132         9      7   161   
 7              2       15             14     194         8      5   174   
 8              3       11             19     218         7     10   101   
 9              3       17              1      80         5     10   203   
 10             7       18             15     147      

# Start generating oof_train, oof_test,  layer1 estimater prediction and model data id list

In [24]:
# uncomment any model to add to the model pool

model_pool = {}

SEED = 2018 

# from sklearn.linear_model import LogisticRegression
# ###################################  Logreg normal   ####################################
# logreg_ble = SklearnBLE(LogisticRegression, nb=False, seed=SEED)
# model_pool[ModelName.LOGREG] = logreg_ble
# ###################################  Logreg nb       ####################################
# nblogreg_params = {'C':0.25}
# nblogreg_ble = SklearnBLE(LogisticRegression, nb=True, seed=SEED, params=nblogreg_params)
# model_pool[ModelName.NBLOGREG] = nblogreg_ble
# ###################################  Logreg per label  ##################################
# logreg_params_per_label = {
#     'identity_hate': {'C': 0.25, 'class_weight': 'balanced', 'fit_intercept': True},
#     'insult': {'C': 0.25, 'class_weight': 'balanced', 'fit_intercept': True},
#     'obscene': {'C': 0.7, 'class_weight': 'balanced', 'fit_intercept': True},
#     'severe_toxic': {'C': 0.3,'class_weight': None, 'fit_intercept': True},
#     'threat': {'C': 0.05, 'class_weight': 'balanced', 'fit_intercept': True}, 
#     'toxic': {'C': 0.8, 'class_weight': 'balanced', 'fit_intercept': True}
# }
# logreg_per_label_ble = SklearnBLE(LogisticRegression, nb=False, seed=SEED, per_label_params=logreg_params_per_label)
# model_pool[ModelName.LOGREG_PERLABEL] = logreg_per_label_ble



# from sklearn.svm import LinearSVC
# ###################################  Logreg normal   ####################################
# lsvc_ble = SklearnBLE(LinearSVC, need_calibrated_classifier_cv=True)
# model_pool[ModelName.LSVC] = lsvc_ble
# ###################################  Logreg nb   ####################################
# nblsvc_params = {'C':0.02}
# nblsvc_ble = SklearnBLE(LinearSVC, nb=True, seed=SEED, params=nblogreg_params, need_calibrated_classifier_cv=True)
# model_pool[ModelName.NBLSVC] = nblsvc_ble


# from xgboost import XGBClassifier
# ###################################  XGB normal   ####################################
# xgb_params = {'n_jobs': 10}
# xgb_ble = SklearnBLE(XGBClassifier, seed=SEED, params=xgb_params)# XGBoostBLE(params=xgb_params, nb=False, seed=SEED)
# model_pool[ModelName.XGB] = xgb_ble
# ###################################  XGB nb   ####################################
# nbxgb_ble = SklearnBLE(XGBClassifier, nb=True, seed=SEED, params=xgb_params)
# model_pool[ModelName.NBXGB] = nbxgb_ble



from lightgbm import LGBMClassifier
###################################  LGB normal   ####################################
lgb_params = {'n_jobs': 3,'n_estimators': 50}
lgb_ble = SklearnBLE(LGBMClassifier, seed=SEED, params=lgb_params)
model_pool[ModelName.LGB] = lgb_ble
# ###################################  LGB nb   ####################################
# nblgb_ble = SklearnBLE(LGBMClassifier, nb=True, seed=SEED, params=lgb_params)
# model_pool[ModelName.NBLGB] = nblgb_ble
# ###################################  LGB per label   ####################################
# lgb_params_per_label = {}
# lgb_params_per_label['toxic'] = {'n_jobs': 8, 'num_leaves': 61}
# lgb_params_per_label['severe_toxic'] = {'n_jobs': 8, 'num_leaves': 11}
# lgb_params_per_label['obscene'] = {'n_jobs': 8, 'num_leaves': 61}
# lgb_params_per_label['threat'] = {'n_jobs': 8, 'num_leaves': 11}
# lgb_params_per_label['insult'] = {'n_jobs': 8,'num_leaves': 11}
# lgb_params_per_label['identity_hate'] = {'n_jobs': 8, 'num_leaves': 61}
# nblgb_per_label_ble = SklearnBLE(LGBMClassifier, seed=SEED, per_label_params=lgb_params_per_label)
# model_pool[ModelName.NBLGB_PERLABEL] = nblgb_per_label_ble

In [25]:
list(model_pool)

[<ModelName.LGB: 5>]

In [14]:
type(train.head(2)['id'].values)

numpy.ndarray

In [65]:
layer1_est_preds, layer1_oof_train, layer1_oof_mean_test, model_data_id_list = compute_layer1_oof(bldr, model_pool, label_cols, sfm_threshold=None)

Computing... label: label        model_data_id: ModelName.LGB_flight_data_1
> /home/kai/data/shiyi/AlphaBoosting/automl/automl_libs/base_layer_utils.py(757)compute_layer1_oof()
-> NFOLDS = 4 # set folds for out-of-fold prediction
(Pdb) type(x_train)
<class 'pandas.core.frame.DataFrame'>
(Pdb) type(y_train)
<class 'list'>
(Pdb) y.shape
*** NameError: name 'y' is not defined
(Pdb) type(x_test)
<class 'pandas.core.frame.DataFrame'>
(Pdb) q


BdbQuit: 

## submit the layer 1 estimator predictions. If they look fine, save them to BaseLayerResultsRepo

In [16]:
list(layer1_est_preds)

['ModelName.LOGREG_PERLABEL_tfidf_word_(1, 1)_30000_1_1.0',
 'ModelName.LOGREG_tfidf_word_(1, 1)_30000_1_1.0',
 'ModelName.NBLOGREG_tfidf_word_(1, 1)_30000_1_1.0']

### generate files to submit from layer 1 estimators

In [22]:
def write_predictions_to_file(base_layer_est_preds):
    for key in base_layer_est_preds:
        submission = pd.read_csv(PATH + 'sample_submission.csv')#.head(1000)
        submission[label_cols] = base_layer_est_preds[key]
        sub_id = int(time.time())
        print(sub_id)
        submission.to_csv('./BaseEstPreds/' + key + '_' + str(sub_id) + '.csv.gz', index=False, compression='gzip')

In [23]:
write_predictions_to_file(layer1_est_preds)

1522094161
1522094166
1522094171


# sanity check before save to BaseLayerResultsRepo

In [25]:
model_data_id_list # model_data we just computed in the layer 1

['ModelName.LOGREG_PERLABEL_tfidf_word_(1, 1)_30000_1_1.0',
 'ModelName.LOGREG_tfidf_word_(1, 1)_30000_1_1.0',
 'ModelName.NBLOGREG_tfidf_word_(1, 1)_30000_1_1.0']

In [27]:
list(layer1_oof_train) # list keys (which are labels)

['obscene', 'identity_hate', 'severe_toxic', 'threat', 'insult', 'toxic']

In [28]:
len(layer1_oof_train['toxic']) # number of models to stack (each model will predict one set of toxic, servere_toxic, etc..)

3

In [29]:
len(layer1_oof_train['toxic'][0]) # examples in oof_train (meta features, x_train) (meta labels are in train[label])

159571

In [31]:
list(layer1_oof_mean_test)

['obscene', 'identity_hate', 'severe_toxic', 'threat', 'insult', 'toxic']

In [32]:
len(layer1_oof_mean_test['toxic'][0]) # examples in oof_test (will be used by meta model (after validation) to predict the final prediction)

153164

In [33]:
# read the doc string to BaseLayerResultsRepo to set params
base_layer_results_repo = BaseLayerResultsRepo(load_from_file=False, filepath='obj/WithPreprocessedFile/')

In [34]:
base_layer_results_repo.add(layer1_oof_train, layer1_oof_mean_test, layer1_est_preds, model_data_id_list)

In [35]:
scores = base_layer_results_repo.show_scores()

0	ModelName.NBLOGREG_tfidf_word_(1, 1)_30000_1_1.0
0	ModelName.LOGREG_tfidf_word_(1, 1)_30000_1_1.0
0	ModelName.LOGREG_PERLABEL_tfidf_word_(1, 1)_30000_1_1.0


In [36]:
 # let's give some fake scores
base_layer_results_repo.add_score('ModelName.NBLOGREG_tfidf_word_(1, 1)_30000_1_1.0', 0.9888)
base_layer_results_repo.add_score('ModelName.LOGREG_tfidf_word_(1, 1)_30000_1_1.0', 0.9777)
base_layer_results_repo.add_score('ModelName.LOGREG_PERLABEL_tfidf_word_(1, 1)_30000_1_1.0', 0.9666)

ModelName.NBLOGREG_tfidf_word_(1, 1)_30000_1_1.0 already existed in the repo. score: 0 update to 0.9888
ModelName.LOGREG_tfidf_word_(1, 1)_30000_1_1.0 already existed in the repo. score: 0 update to 0.9777
ModelName.LOGREG_PERLABEL_tfidf_word_(1, 1)_30000_1_1.0 already existed in the repo. score: 0 update to 0.9666


In [37]:
scores = base_layer_results_repo.show_scores()

0.9888	ModelName.NBLOGREG_tfidf_word_(1, 1)_30000_1_1.0
0.9777	ModelName.LOGREG_tfidf_word_(1, 1)_30000_1_1.0
0.9666	ModelName.LOGREG_PERLABEL_tfidf_word_(1, 1)_30000_1_1.0


In [38]:
base_layer_results_repo.save()

In [17]:
# Next step: go to stack_layer2

In [18]:
print('stack 1')

stack 1
